<img style="float: left; padding-right: 20px; width: 70px" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/1920px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"> 

**Pontificia Universidad Católica de Chile**<br>
**Semestre 2021-2**<br>
**Profesora:** Paula Aguirre <br>

----
<div style="background-color:#fffff;">
    <br>
    <p style="text-align:center;font-family:Times New Roman;color:black;font-size:20pt;font-weight:bold;margin:20px">
        Introducción a la Ciencia de Datos: Proyecto Final
    </p>
    <p style="text-align:center;font-family:Times New Roman;color:black;font-size:20pt;font-weight:bold;margin:20px">
        Influencia de los factores sociales y económicos en la proyección de la
delincuencia.
    </p>
    <p style="text-align:center;font-family:Times New Roman;color:black;font-size:12pt;font-weight:bold;margin:20px">
        <br>Cristóbal Cancino - cristobal.cancinob@uc.cl</br>
        <br>Diego Rodríguez - drodrguez@uc.cl</br>
        <br>Benjamín Prieto - bprietz@uc.cl</br>
        <br>Larry Uribe - larry@uc.cl</br>
        <br>14 de diciembre de 2021
        <br><br><b>Tiempo de ejecución aproximado:</b> x minutos.
    </p>
    <br><br>
</div>

## El proyecto se divide en 7 etapas: 

1. Extracción de datos
        data_total_dmcs -> Delito
        data_educacion -> calidad educacion
        data_pobreza -> indicadores de pobreza
        data_comunas -> info de comunas
        data_natalidad -> info de natalidad (diccionario por años) 
        data_mortalidad -> info de mortalidad (diccionario por años)
        data_salud -> establecimientos MINSAL (mirar años)


2. Limpieza de datos

3. Procesamiento y transformación

4. Analisis exploratorio

5. Aprendizaje supervisado
        regresiones multilineal, ridge, lasso -> a predecir el total_dmcs

6. Aprendizaje no supervisado
        aplicaremos kmeans sobre mapa de chile o santiago (definir comunas) (puede ser uno  por pais y region, y otro por comunas de santiago)

7. Conclusiones y visualizaciones al cierre. 

## 1. Extraccion de datos:

In [1]:
import pandas as pd
import requests 
import os
import io, zipfile

In [3]:
#Cargar archivos (I) : Data sobre denuncias de crimen de mayor connotación social.

link_denuncias = 'http://cead.spd.gov.cl/centro-de-documentacion/?wpdmpro=estadisticas-anuales-delitos-de-mayor-connotacion-social-dmcs-base-censo-2017&wpdmdl=3047&'
rdenuncias = requests.get(link_denuncias, stream = True)

#Se necesita crear la carpeta para almacenar los archivos de datos.
#exist_ok permite evitar errores derivados de la ya-existencia de la carpeta.
ruta_delitos = os.path.join('Data')
os.makedirs(ruta_delitos, exist_ok = True)

#Descompresión del archivo necesario de todo el ZIP
z = zipfile.ZipFile(io.BytesIO(rdenuncias.content))
z.extractall(path=ruta_delitos, members=['Frecuencia Anual_DENUNCIA.xlsx'] )

#Crea dataframe
path_denuncias = os.path.join('Data', 'Frecuencia Anual_DENUNCIA.xlsx')
data_total_dmcs = pd.read_excel(path_denuncias, sheet_name = 'TOTAL_DMCS')
#columnas_denuncias = data_denuncias.keys()

# Carga archivos (II) : Data de calidad de educación.
link_educacion = 'http://datos.mineduc.cl/datasets/193132-bd-sned-2018-2019.download/'
data_educacion = pd.read_csv(link_educacion, delimiter = ';')

# Carga archivos (III): Data de indicador pobreza

link_pobreza = 'http://observatorio.ministeriodesarrollosocial.gob.cl/storage/docs/pobreza-comunal/2017/PLANILLA_Estimaciones_comunales_tasa_pobreza_por_ingresos_multidimensional_2017.xlsx'
data_pobreza = pd.read_excel(link_pobreza, sheet_name = 'Multidimensional 2017', engine='openpyxl')

# Cargar archivos (IV) : Información varía sobre comunas.
link_info_comunal = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/input/Otros/InformacionComunas.csv"
data_comunas = pd.read_csv(link_info_comunal)

# Cargar archivos (V) : Información sobre natalidad se genera dinamicamente lista de archivos
datas_natalidad = dict()

for ano in range(2011, 2021):
    link_natalidad_p = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/input/RegistroCivil/Nacimientos/Nacimientos_"+str(ano)+"-01-01_"+str(ano)+"-12-31_DO.csv"
    datas_natalidad[ano] = pd.read_csv(link_natalidad_p)
    
# Cargar archivos (VI): Información sobre la mortalidad. se carga dinamico por año

datas_mortalidad = dict()

for ano in range(2011, 2021):
    link_mortalidad_p = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/input/RegistroCivil/Defunciones/Defunciones_"+str(ano)+"-01-01_"+str(ano)+"-12-31_DO.csv"
    datas_mortalidad[ano] = pd.read_csv(link_mortalidad_p)

# Cargar archivos (VII) : Información sobre acceso a la salud
# Para medir el acceso a la salud solo se considerará la información actualizada al 2021, por el acceso a los datos;
# Y también por la poca variabilidad del nivel de hospitales

link_salud = 'https://repositoriodeis.minsal.cl/DatosAbiertos/Establecimientos_ChileDEIS_MINSAL%2029-10-2021.xlsx'
data_salud = pd.read_excel(link_salud, sheet_name = 'Establecimientos V')

HTTPError: HTTP Error 403: Forbidden

## 2. Limpieza de datos:

In [ ]:
# Conjunto de datos I: Delitos de mayor connotación social
display(data_total_dmcs.head())
display(data_total_dmcs.info())

In [ ]:
#Reestructuración y limpieza
data_total_dmcs = data_total_dmcs[1:]
data_total_dmcs.columns = data_total_dmcs.iloc[1]
data_total_dmcs = data_total_dmcs[data_total_dmcs['UN_ADMIN'] == 'COMUNA'] 
data_total_dmcs = data_total_dmcs[['UNIDAD', 2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0]]
#revisar como eliminar el header del index
data_total_dmcs.reset_index(level=0, inplace=True, drop=True)
data_total_dmcs.rename(columns={'UNIDAD': 'Comuna', 2011.0: '2011', 2012.0: '2012', 2013.0: '2013', 2014.0: '2014', 2015.0: '2015', 2016.0: '2016', 2017.0: '2017', 2018.0: '2018', 2019.0: '2019', 2020.0: '2020'}, inplace=True)
display(data_total_dmcs.head())
display(data_total_dmcs.info())


In [ ]:
#Definición de nuevas columnas
data_total_dmcs['Total delitos'] = data_total_dmcs.sum(axis = 1, numeric_only = True)
data_total_dmcs['Delitos promedio'] = data_total_dmcs['Total delitos'] / len(range(2011, 2021))
#Restricción a las columnas necesarias
data_total_dmcs = data_total_dmcs[['Comuna', 'Total delitos', 'Delitos promedio']]
data_total_dmcs.head()

In [ ]:
#Conjunto de datos II: Calidad de la educación 
display(data_educacion.head())
display(data_educacion.info())

In [ ]:
#Reestructuración y limpieza
data_educacion = data_educacion[['NOM_COM_RBD', 'INDICER']]
data_educacion["INDICER"]=data_educacion["INDICER"].str.replace(',', '.').astype(float)
data_educacion.rename(columns={'NOM_COM_RBD': 'Comuna', 'INDICER': 'Calidad Educacion'}, inplace=True)
data_educacion = data_educacion.groupby(by=["Comuna"]).mean()
data_educacion = data_educacion.reset_index()
data_educacion.head()

In [ ]:
data_educacion['Comuna'] = data_educacion['Comuna'].str.title()

In [ ]:
data_educacion.head()

In [ ]:
data_educacion.info()

In [ ]:
#Conjunto de datos III: Pobreza multidimensional
data_pobreza.head()

In [ ]:
header_row = 1
data_pobreza.columns = data_pobreza.iloc[header_row]
data_pobreza = data_pobreza[2:]
data_pobreza.info()

In [ ]:
data_pobreza = data_pobreza[['Nombre comuna', 'Porcentaje de personas en situación de pobreza multidimensional 2017']]
data_pobreza.rename(columns = {'Nombre comuna': 'Comuna', 'Porcentaje de personas en situación de pobreza multidimensional 2017': 'Porcentaje de pobreza'}, inplace = True)
data_pobreza['Porcentaje de pobreza'] = data_pobreza['Porcentaje de pobreza'] * 100
data_pobreza.head()

In [ ]:
#Conjunto de datos IV: Información sobre población y superficie comunal.
display(data_comunas.head())
display(data_comunas.info())

In [ ]:
#Reestructuración
data_comunas = data_comunas[['Comuna', 'Superficie_km2', 'Poblacion']]
data_comunas.head()

In [ ]:
#Conjunto de datos V: Sobre natalidad anual por comuna
#Reestructuración iterativa de los datos
#Disclaimer: La siguiente reestructuración se hace en conocimiento de la estructura compartida de los datasets.

for ano in range(2011, 2021):
    datas_natalidad[ano] = datas_natalidad[ano][['Comuna', 'Nacimientos']]
    datas_natalidad[ano] = datas_natalidad[ano].groupby(by = 'Comuna').sum()
    datas_natalidad[ano].reset_index(inplace = True)
    if ano == 2011:
        resumen_natalidad = datas_natalidad[2011]
    else:
        resumen_natalidad = resumen_natalidad.merge(datas_natalidad[ano], on = 'Comuna', how = 'outer')


In [ ]:
display(resumen_natalidad.head())
display(resumen_natalidad.info())


#SPOILER: no funciona

In [ ]:
#Conjunto de datos VI: Número de establecimientos de salud por comuna
display(data_salud.head())
display(data_salud.info())

In [ ]:
header_row = 0
data_salud.columns = data_salud.iloc[header_row]
data_salud.head()

In [ ]:
data_salud.info()

In [ ]:
data_salud_aux = data_salud[['Nombre Comuna']]
data_salud_aux['Recintos Salud'] = 1
data_salud_aux = data_salud_aux.groupby(by = 'Nombre Comuna').count().reset_index()
data_salud_aux.rename(columns = {'Nombre Comuna': 'Comuna'}, inplace = True)
display(data_salud_aux.head())
display(data_salud_aux.info())

In [ ]:
# PRUEBAS DE FUNCIONAMIENTO DE MERGE
data_1 = pd.merge(left = data_comunas, right = data_salud_aux, how = 'inner', on = 'Comuna')

In [ ]:
data_2 = data_1.merge(data_educacion, how = 'inner', on = 'Comuna')

In [ ]:
data_3 = data_2.merge(data_total_dmcs, how = 'inner', on = 'Comuna')

In [ ]:
data_3

In [ ]:
data_4 = data_3.merge(data_pobreza, how = 'inner', on = 'Comuna')
data_4.head()

In [ ]:
data_4.corr()['Delitos promedio']

In [ ]:
data_4['Porcentaje de pobreza'] = data_4['Porcentaje de pobreza'].astype(float)

In [ ]:
data_4.corr()['Delitos promedio']

In [ ]:
data_4['Poblacion por km2'] = data_4['Poblacion'] /  data_4['Superficie_km2']
data_4.corr()

In [ ]:
data_4['Delitos por habitante'] = data_4['Delitos promedio'] / data_4['Poblacion']
data_4.corr()['Delitos por habitante']

In [ ]:
data_4.head()